In [1]:
import cv2
import numpy as np
from img_util import get_saliency_hist, mapping_pallet_to_img
from btpd import BTPD_WTSE_LimitationSv, BTPD_WTSE
from proposal import CIQ_test, get_img_importance, get_importance_error, ciq_eval_set

In [4]:
def find_closest_color(pix, source):
    tile = np.tile(pix, len(source)).reshape(source.shape)
    dist = np.linalg.norm(tile - source, axis=2)
    index = np.argmin(dist)
    return source[index]


def FloydSteinbergDithering(img, palette):
    height, width = img.shape[0], img.shape[1]
    org = img.copy()
    for y in range(height):
        for x in range(width):
            closest = find_closest_color(org[y, x], palette)
            org[y, x] = closest
            q_err = org[y, x] - closest
            
            if x < width - 1:
                org[y, x + 1] = org[y, x + 1] + (7.0 * q_err) / 16
            if y < height - 1:
                org[y + 1, x] = org[y + 1, x] + (5.0 * q_err) / 16
                if x > 0:
                    org[y + 1, x - 1] = org[y + 1, x - 1] + (3.0 * q_err) / 16
                if x < width - 1:
                    org[y + 1, x + 1] = org[y + 1, x + 1] + q_err / 16.0
    return org


def CIQ_test_BTPD_WithImpoertance(M=[16], DIR=['sumple_img']):
    """
    重要度を，合計顕著度ではなく，その色の中で最大の顕著度とする
    :param M:
    :param DIR:
    :param PRE_Q:
    :param DIV:
    :return:
    """
    test_config = {
        'trans_flag': True,
        'trans_code': cv2.COLOR_BGR2LAB,
        'trans_inverse_code': cv2.COLOR_LAB2BGR,
        'view_distribution': True,
        'save_tmp_imgs': False,
        'view_importance': True,
        'importance_eval': get_importance_error,
        'ciq_error_eval': ciq_eval_set(),
        'mapping': mapping_pallet_to_img
    }
    for dir in DIR:
        for m in M:
                code = cv2.COLOR_BGR2LAB
                inverse_code = cv2.COLOR_LAB2BGR

                def ciq(img, **ciq_status):
                    num = ciq_status['num']
                    name = ciq_status['img_path']
                    trans_img = cv2.cvtColor(img, code)
                    # trans_img = img.copy()
                    S = np.reshape(trans_img, newshape=(img.shape[0] * img.shape[1], 3)).astype(np.uint64)

                    # SM count in each colors
                    Sv_map = get_img_importance(name, statistics='sum_imp').values.flatten()
                    uniq_S = np.unique(S, axis=0)
                    # tmp = S == uniq_S[0]
                    # tmp = np.all(S == uniq_S[0], axis=1)
                    # tmp_pix = S[np.where(np.all(S == uniq_S[0], axis=1))[0]]
                    # tmp = Sv_map[np.where(np.all(S == uniq_S[0], axis=1))[0]]
                    uniq_Sv = np.array([np.mean(Sv_map[np.where(S == color)[0]]) for color in uniq_S]).astype(np.float32)

                    q, root, groups = BTPD_WTSE(uniq_S, m, uniq_Sv)

                    # importances = uniq_S[np.argsort(uniq_Sv)]
                    dict = {'palette': q,
                            'groups': groups,
                            'tmp_sm': Sv_map}
                    return dict

                SAVE = 'BTPD_WithImportance_m{}_{}_LAB_nodither'.format(m, dir)
                CIQ_test(ciq, SAVE, test_img=dir, **test_config)
            
            
def CIQ_test_ProposalSvSumWeight(M=[16], DIR=['sumple_img'], LIMIT=[3000]):
    """
    重要度を，合計顕著度ではなく，その色の中で最大の顕著度とする
    :param M:
    :param DIR:
    :param PRE_Q:
    :param DIV:
    :return:
    """
    test_config = {
        'trans_flag': True,
        'trans_code': cv2.COLOR_BGR2LAB,
        'trans_inverse_code': cv2.COLOR_LAB2BGR,
        'view_distribution': True,
        'save_tmp_imgs': True,
        'view_importance': False,
        'importance_eval': False,
        'ciq_error_eval': ciq_eval_set(), 
        'mapping': FloydSteinbergDithering
    }
    for dir in DIR:
        for m in M:
            for lim in LIMIT:
                def ciq(img, **ciq_status):
                    trans_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
                    # trans_img = img.copy()
                    org_S = np.reshape(img, newshape=(img.shape[0] * img.shape[1], 1, 3)).astype(np.uint64)
                    S = np.reshape(trans_img, newshape=(img.shape[0] * img.shape[1], 1, 3)).astype(np.uint64)
                    _, __, Sv_map = get_saliency_hist(trans_img, sm='SR')
                    org_Sv = np.reshape(Sv_map, newshape=(len(S), 1, 1)).astype(np.float32) / np.max(Sv_map)
                    # pre quantize
                    pre_q, root, pre_groups = BTPD_WTSE_LimitationSv(S, org_Sv, lim)
                    pre_mapped = mapping_pallet_to_img(trans_img, pre_q)
                    print(f'{len(np.unique(org_S, axis=0))}')
                    # SM count in each colors
                    _, __, Sv_map = get_saliency_hist(pre_mapped, sm='SR')
                    Sv = np.reshape(Sv_map, newshape=(len(S), 1, 1)).astype(np.float32) / np.max(Sv_map)
                    S = np.reshape(pre_mapped, newshape=(len(S), 1, 3)).astype(np.uint64)
                    uniq_S = np.unique(S, axis=0)
                    uniq_Sv = [Sv[np.where(color == S)[0]] for color in uniq_S]
                    # len_max = np.max([len(sv_array) for sv_array in uniq_Sv])
                    # uniq_Sv = np.array([np.mean(sv_array) * (np.sum(sv_array) / len_max) for sv_array in uniq_Sv])
                    uniq_Sv = np.array([np.sum(sv_array) for sv_array in uniq_Sv])
                    # only in case of sum
                    print('pre quantize {} colors'.format(len(root.get_leaves())))
                    q, root, groups = BTPD_WTSE(uniq_S, m, uniq_Sv)
                    pre_mapped = cv2.cvtColor(pre_mapped, cv2.COLOR_LAB2BGR)
                    reshape_q = np.reshape(q, newshape=(m, 1, 3)).astype(np.uint8)
                    retrans_q = cv2.cvtColor(reshape_q, cv2.COLOR_LAB2BGR)
                    reshape_pre_q = np.reshape(pre_q, newshape=(len(pre_q), 1, 3)).astype(np.uint8)
                    retrans_pre_q = cv2.cvtColor(reshape_pre_q, cv2.COLOR_LAB2BGR)
                    dict = {'palette': q,
                            'groups': [retrans_pre_q[:, 0, :], retrans_q[:, 0, :]],
                            'save_imgs': [{'img': Sv_map, 'filename': 'tmp_Sv.jpg'},
                                          {'img': pre_mapped, 'filename': 'pre_mapped.jpg'}]}
                    return dict
                SAVE = 'ProposalSvSumWeight_m{}_{}_lim{}_Dither'.format(m, dir, lim)
                CIQ_test(ciq, SAVE, test_img=dir, **test_config)

In [5]:
print('start test')
# CIQ_test_ProposalSvSumWeight(M=[16, 32], DIR=['sumple_img'], LIMIT=[1000])
CIQ_test_BTPD_WithImpoertance(M=[16, 32], DIR=['sumple_img'])

start test
Aerial.bmp , by BTPD_WithImportance_m16_sumple_img_LAB_nodither, calc time 41.94378399848938s
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\2340colors_distribution_n1.jpg
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\815colors_distribution_n2.jpg
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\1379colors_distribution_n3.jpg
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\2889colors_distribution_n4.jpg
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\2588colors_distribution_n5.jpg
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\1514colors_distribution_n6.jpg
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\2686colors_distribution_n7.jpg
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\2970colors_distribution_n8.jpg
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\2939colors_distribution_n9.jpg
save BTPD_WithImportance_m16_sumple_img_LAB_nodither\Aerial\3344colors_distri